In [1]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model
from keras import backend
from keras import callbacks
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import cv2
import matplotlib.image as mpimg
from random import shuffle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

Using TensorFlow backend.


In [2]:
backend.image_dim_ordering()

'tf'

In [3]:
def make_model_vgg16():
    # vgg16
    model_vgg16_conv = VGG16(weights='imagenet', include_top=False)

    #Create input format)
    input = Input(name = 'image_input, shape=(224,224,3))

    #Use the generated model 
    output_vgg16_conv = model_vgg16_conv(input)

    # freeze vgg16 conv layers
    for layer in model_vgg16_conv.layers:
        layer.trainable = False

    #model_vgg16_conv.summary()

    # Add the fully-connected layers 
    x = Flatten(name='flatten')(output_vgg16_conv)
    x = Dense(256, activation='relu', name='fc1')(x)
    x = Dense(256, activation='relu', name='fc2')(x)
    x = Dense(1, activation='linear', name='predicton_steering')(x)

    # Create model 
    return Model(input=input, output=x)

In [ ]:
def make_model_nvidia():
    # based on this paper:
    # http://images.nvidia.com/content/tegra/automotive/images/2016/solutions/pdf/end-to-end-dl-using-px.pdf
    input = Input(shape=(224,224,3),name = 'image_input')

    x = Conv2D(name='conv1', 24, 5, 5, border_mode="valid", subsample=(2, 2), activation="relu")(input)
    x = Conv2D(name='conv2'36, 5, 5, border_mode="valid", subsample=(2, 2), activation="relu")(x)
    x = Conv2D(name='conv3'48, 5, 5, border_mode="valid", subsample=(2, 2), activation="relu")(x)
    x = Conv2D(name='conv4'64, 3, 3, border_mode="valid", subsample=(1, 1), activation="relu")(x)
    x = Conv2D(name='conv5'64, 3, 3, border_mode="valid", subsample=(1, 1), activation="relu")(x)
    x = Flatten(name='flatten')(x) 
    x = Dense(1164, activation="relu")(x))
    x = Dense(100, activation="relu")(x)
    x = Dense(50, activation="relu")(x)
    x = Dense(10, activation="relu")(x)
    x = Dense(1, activation="linear")(x)
    
    # Create model
     return Model(input=input, output=x)

In [4]:
my_model=make_model()

58892288/58889256 [==============================] - 3s     


In [5]:
base_path = os.getcwd()

In [6]:
def preprocess_data_log(base_path, csv_file_name, split=0.2):
    # read log file
    log = pd.read_csv(base_path + '/' + csv_file_name)
    # modify img file name column to contain the full path
    log['center'] = base_path + '/'+ log['center']
    # extract img file names
    img_names = log['center'].tolist()
    # extract angles
    angles = log['steering']
    
    img_names_train, img_names_test, angles_train, angles_test = train_test_split(img_names, angles, test_size=split, random_state=42)
    
    return ( list(zip(img_names_train, angles_train)), list(zip(img_names_test, angles_test)) )

In [9]:
(train_data_list, test_data_list) = preprocess_data_log(base_path + '/data/udacity/data', 'driving_log.csv')
#train_data_list[0:3]

In [10]:
my_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

In [11]:
def batch_gen(name, data_list, out_img_size, batch_size):
    # data_list is in this form: ['file_name', angle]
    # out_img_size is a shape of the output image: (x, y, color)
    
    # create batch_size np arrays as placeholders for imgages and angles
    X_train = np.empty((batch_size,) +  out_img_size, dtype = np.float64)
    y_train = np.empty(batch_size, dtype = np.float32)
    
    while True:
        # shuffle data
        shuffle(data_list)
        for offset in range(0, len(data_list), batch_size):
            end = offset + batch_size
            batch_subset = data_list[offset:end]
            for i, _ in enumerate(batch_subset):
                #load image
                X_train[i] = cv2.resize(cv2.imread(batch_subset[i][0]), (224, 224)).astype(np.float32) - 128.
                #X_train[i] = im.transpose((2,0,1))
                #X_train[i] = np.expand_dims(im, axis=0)
                # load angle
                y_train[i] = batch_subset[i][1]
                
            batch_X, batch_y = X_train[:len(batch_subset)], y_train[:len(batch_subset)]
            #print(name)
            yield (batch_X, batch_y)

In [12]:
#g=batch_gen(out[0:6], (160, 320, 3), 2)

In [13]:
#(im, a) = next(g)
len(train_data_list)

6428

In [14]:
len(test_data_list)

1608

In [15]:
#plt.imshow(cv2.cvtColor((im[0]*255.+128.).astype(np.uint8), cv2.COLOR_BGR2RGB))
#im0 = cv2.resize(cv2.imread(train_data_list[0][0]), (224, 224))
#plt.imshow(cv2.cvtColor(im0, cv2.COLOR_BGR2RGB))

In [16]:
class printbatch(callbacks.Callback):
    def on_batch_end(self, epoch, logs={}):
        print(logs)
        
pb = printbatch()

In [22]:
train_gen = batch_gen("train_gen", train_data_list, (224, 224, 3), 64)
test_gen = batch_gen("test_gen", test_data_list, (224, 224, 3), 64)

In [23]:
history = my_model.fit_generator(
    generator=train_gen, 
    validation_data=test_gen, 
    nb_val_samples=len(test_data_list), 
    samples_per_epoch=len(train_data_list), 
    nb_epoch=10,
    max_q_size=1)

Epoch 1/10
6428/6428 [==============================] - 266s - loss: 21.6338 - acc: 0.1024 - val_loss: 0.9705 - val_acc: 0.2239
Epoch 2/10
6428/6428 [==============================] - 253s - loss: 1.2239 - acc: 0.1979 - val_loss: 0.9961 - val_acc: 0.2276
Epoch 3/10
6428/6428 [==============================] - 253s - loss: 0.8866 - acc: 0.2178 - val_loss: 0.7317 - val_acc: 0.2338
Epoch 4/10
6428/6428 [==============================] - 253s - loss: 1.1860 - acc: 0.1934 - val_loss: 1.2134 - val_acc: 0.1561
Epoch 5/10
6428/6428 [==============================] - 253s - loss: 5.2690 - acc: 0.1335 - val_loss: 2.5210 - val_acc: 0.1275
Epoch 6/10
6428/6428 [==============================] - 253s - loss: 2.6741 - acc: 0.1520 - val_loss: 1.9546 - val_acc: 0.0435
Epoch 7/10
6428/6428 [==============================] - 253s - loss: 0.8192 - acc: 0.2307 - val_loss: 0.6778 - val_acc: 0.1915
Epoch 8/10
6428/6428 [==============================] - 253s - loss: 1.0492 - acc: 0.2288 - val_loss: 1.3132 -